In [1]:
# dlwpt chapter 3: 
# it starts with a tensor
# Understanding tensors
# indexing and operating on tensors
# interoperating with numpy multi dim arrays
# moving computations to the GPU for speed

In [2]:
# starting off with regular python list indexing
a = [1.0, 2.0, 1.0]

In [3]:
a[0]

1.0

In [4]:
a[2] = 3.0
a[2]

3.0

In [5]:
a

[1.0, 2.0, 3.0]

In [6]:
import torch

In [7]:
a = torch.ones(3)
a

tensor([1., 1., 1.])

In [8]:
a[0]

tensor(1.)

In [9]:
float(a[1])

1.0

In [10]:
a[2] = 2
a

tensor([1., 1., 2.])

In [11]:
# using a 2d tensor to represent coordinates
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [12]:
points.shape

torch.Size([3, 2])

In [13]:
points[1]

tensor([5., 3.])

In [14]:
points[1:]

tensor([[5., 3.],
        [2., 1.]])

In [15]:
points[1:, :]

tensor([[5., 3.],
        [2., 1.]])

In [16]:
points[1:, 0]

tensor([5., 2.])

In [17]:
new_points = points[None]
new_points.size()

torch.Size([1, 3, 2])

In [18]:
# trying out image transform to greyscale with made up data
img_t = torch.randn(3, 5, 5)
weights = torch.tensor([0.2126, 0.7152, 0.0722])

In [19]:
batch_t = torch.randn(2,3,5,5) # shape[batch, channels, rows, columns]

In [20]:
# the RGB channels are in dimension 0 or sometimes in dimension 1. But they are always in dimension -3 
# if counted from the end 
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [21]:
# broadcasting appends leading dimensions of size 1 automatically
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)
unsqueezed_weights.shape
img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [22]:
# this gets messy very quickly
# the pytorch function einsum specifies an indexing mini language (einstein sum notation)
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weights)
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weights)
batch_gray_weighted_fancy.shape

torch.Size([2, 5, 5])

In [23]:
# this gets messy rather quickly still, so we can name tensor dimensions to keep everything tidy
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named

<ipython-input-23-958a076494c5>:2: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:840.)
  weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [24]:
# we can combine the refine function and the ellipsis operator to rename dimensions
img_named =  img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [32]:
# we still need to align dimensions so to do this explicitly we can use the method align_as
weights_aligned = weights_named.align_as(img_named)
weights_named.shape, weights_aligned.shape

(torch.Size([3]), torch.Size([3, 1, 1]))

In [34]:
# functions accepting dimension arguments also take named dims
gray_named = (img_named * weights_aligned).sum("channels")
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [41]:
# to remove names we can just rename to None
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))

In [44]:
# we can specify the numeric type of a tensor with the dtype argument
# tensors can be used as indices in other tensors
# to accomplish this we create tensors with integers as arguments
# this yields an int64 tensor by default
# predicates on tensors such as points > 1.0 produce bool tensors 
# managing data types: 
double_points = torch.ones(10, 2, dtype=torch.double)
short_points = torch.tensor([[1, 2], [3, 4]], dtype=torch.short)
short_points.dtype, double_points.dtype

(torch.int16, torch.float64)

In [46]:
# typecasting can be done in two ways
double_points = torch.zeros(10, 2).double()
short_points = torch.ones(10, 2).short()

double_points = torch.zeros(10, 2).to(torch.double)
short_points = torch.ones(10, 2).to(dtype = torch.short)
short_points, double_points

(tensor([[1, 1],
         [1, 1],
         [1, 1],
         [1, 1],
         [1, 1],
         [1, 1],
         [1, 1],
         [1, 1],
         [1, 1],
         [1, 1]], dtype=torch.int16),
 tensor([[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]], dtype=torch.float64))

In [50]:
# short intro to the torch api
a = torch.ones(3,2)
a_t = a.transpose(0,1)
a, a_t

(tensor([[1., 1.],
         [1., 1.],
         [1., 1.]]),
 tensor([[1., 1., 1.],
         [1., 1., 1.]]))

In [51]:
# we get the same result calling the torch function directly
a_t2 = torch.transpose(a, 0, 1)
a, a_t2

(tensor([[1., 1.],
         [1., 1.],
         [1., 1.]]),
 tensor([[1., 1., 1.],
         [1., 1., 1.]]))

In [52]:
# indexing into storage
# tensors are stored as 1D arrays in memory
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points, points.storage()

(tensor([[4., 1.],
         [5., 3.],
         [2., 1.]]),
  4.0
  1.0
  5.0
  3.0
  2.0
  1.0
 [torch.FloatStorage of size 6])

In [54]:
# if we modify the value of a storage we also modify the value of the tensor object 
points.storage()[0] = 10
points

tensor([[10.,  1.],
        [ 5.,  3.],
        [ 2.,  1.]])

In [55]:
# methods with a trailing underscore in their name, like zero_ 
# operate in place by modifying the input instead of creating a new output tensor and returning it
# any method without the trailing underscores returns a new tensor and leaves the original tensor untouched
a = torch.ones(3,2)
a.zero_()

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [61]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point.storage_offset(), second_point.shape

(2, torch.Size([2]))

In [63]:
points.stride()

(2, 1)

In [65]:
# managing a tensors device attribute
# points_gpu = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]], device='cuda')
# not available on my machine because my GPU is too old :(

In [67]:
# we can convert from numpy and back using the functions
points = torch.ones(3,4)
points_np = points.numpy()
points_torch = torch.from_numpy(points_np)
points_np, points_torch, points

(array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32),
 tensor([[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]),
 tensor([[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]))

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)